# Predicting F1 Qualification and Race Outcomes Using Practice Session Data

# Introduction
Formula 1 is a sport where milliseconds matter, and predictive insights can be the difference between victory and defeat. This project leverages data from Free Practice sessions (FP1, FP2, and FP3) to predict the outcome of race day—specifically, the eventual race winner. From a motorsport strategy perspective, we analyze key performance indicators such as lap times, tire compound choices, stint durations, and tire degradation rates to assess how teams and drivers prepare for qualifying and race simulations.

On the data science front, we build a structured pipeline using the FastF1 library to extract, process, and engineer features from historical Grand Prix data. Machine learning models—including classification and regression techniques—are employed to correlate practice performance with final race outcomes. By combining domain-specific race knowledge with predictive analytics, this project aims to anticipate on-track results before the lights go out.

### Importing Libraries

In [1]:
import fastf1
from fastf1.ergast import Ergast
from fastf1 import plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.decomposition import PCA
import shap
import streamlit as st
import warnings

warnings.filterwarnings('ignore')
fastf1.Cache.enable_cache('./fastf1_cache')

# Load Data using legacy Ergast fallback to avoid SSL verification issues
# import urllib3
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# import fastf1.ergast.interface as ergast_interface

C:\Users\aeropc6\.conda\envs\dev_env\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\aeropc6\.conda\envs\dev_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and Extracxt Data

#### Load Session Data

In [2]:
# Load Session Data
def load_session_data(season, gp, session):
    ses = fastf1.get_session(season, gp, session)
    ses.load()
    return ses

In [3]:
season = 2024
gp_name = 'Monaco'
ses = 'FP2'
session = load_session_data(season, gp_name, ses)

core           INFO 	Loading data for Monaco Grand Prix - Practice 2 [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver  3: Lap timing integrity check failed for 1 lap(s)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cac

#### Extracting Practice Data

In [4]:
# Extracting Practice Session Stats
def extract_practice_features(season, gp_name, ses_name):
    session = load_session_data(season, gp_name, ses_name)
    laps = session.laps.pick_quicklaps()
    drivers = session.drivers

    data = []
    for driver in drivers:
    
        driver_laps = laps.pick_driver(driver)
        if driver_laps.empty:
            continue
        fastest_lap = driver_laps['LapTime'].min()
        avg_lap = driver_laps['LapTime'].mean()
        total_laps = len(driver_laps)

        data.append({
            'Driver': session.get_driver(driver)['Abbreviation'],
            f'{ses_name}_FastestLap': fastest_lap.total_seconds(),
            f'{ses_name}_AvgLapTime': avg_lap.total_seconds(),
            f'{ses_name}_NumLaps': total_laps,
            # **tire_compound_counts # e.g. 'Soft': 6, 'Medium': 3
        })

    return pd.DataFrame(data)

In [5]:
def get_practice_data(season, gp_name):
    sessions = ['FP1', 'FP2', 'FP3']
    df = pd.DataFrame()
    for i, sess in enumerate(sessions):
        if df.empty:
            df = extract_practice_features(season, gp_name, sess)
            continue
        drivers = extract_practice_features(season, gp_name, sess)
        df = df.merge(drivers, on='Driver', how='left')

    return df

In [6]:
df =  get_practice_data(season, gp_name)
df

core           INFO 	Loading data for Monaco Grand Prix - Practice 1 [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	

,Driver,FP1_FastestLap,FP1_AvgLapTime,FP1_NumLaps,FP2_FastestLap,FP2_AvgLapTime,FP2_NumLaps,FP3_FastestLap,FP3_AvgLapTime,FP3_NumLaps
0,VER,72.984,74.024285,7,71.813,74.566230,13,71.566,72.518333,9.0
1,SAR,74.150,75.700384,13,72.790,74.474000,10,72.703,73.325000,6.0
2,RIC,72.901,74.957076,13,72.577,74.191800,10,72.829,73.382625,8.0
3,NOR,72.396,74.888800,15,71.953,74.261900,10,71.988,73.117285,7.0
4,GAS,75.574,75.574000,1,72.750,74.268900,10,72.144,73.246333,9.0
5,PER,73.229,74.739571,7,72.099,74.480833,12,71.923,73.339333,6.0
6,ALO,72.775,74.422500,8,71.753,73.006000,7,72.087,73.881428,7.0
7,LEC,72.397,73.831500,10,71.278,72.977454,11,71.369,72.474000,8.0
8,STR,72.789,74.834900,10,72.062,73.013125,8,72.331,73.412700,10.0
9,MAG,73.390,75.451000,16,72.473,74.426750,12,72.216,73.051222,9.0


#### Extract tire degradation features

In [7]:
# Extract tire degradation features
def get_tire_degradation_features(session):
    laps = session.laps
    laps = laps.pick_accurate()
    drivers = session.drivers
    deg_data = []

    for driver in drivers:
        driver_laps = laps.pick_driver(driver).pick_wo_box()
        if driver_laps.empty:
            continue

        for compound in driver_laps['Compound'].unique():
            comp_laps = driver_laps[driver_laps['Compound'] == compound]

            for stint in comp_laps['Stint'].unique():
                stint_laps = comp_laps[comp_laps['Stint'] ==  stint]
                stint_laps = stint_laps[stint_laps['LapTime'].notna()]
                if len(stint_laps) < 4:
                    continue

                trimmed = stint_laps.iloc[1:-1]
                if len(trimmed) < 3:
                    continue
                
                lap_times = trimmed['LapTime'].dt.total_seconds()
                lap_nums = trimmed['LapNumber']
                
                # Regression for degradation rate
                X = lap_nums.values.reshape(-1, 1)
                y = lap_times.values
                slope = LinearRegression().fit(X, y).coef_[0]

                # Degradation Rate
                deg_rate = max(slope, 0)
    
                deg_data.append({
                    'Driver': session.get_driver(driver)['Abbreviation'],
                    'Compound': compound,
                    'DegRate': deg_rate
                })
            
    return pd.DataFrame(deg_data)

In [8]:
tire_deg = get_tire_degradation_features(session)
tire_deg

,Driver,Compound,DegRate
0,VER,HARD,0.775750
1,VER,HARD,0.000000
2,SAR,MEDIUM,0.371631
3,SAR,SOFT,0.482000
4,RIC,HARD,2.547520
5,RIC,MEDIUM,1.225607
6,RIC,MEDIUM,0.000000
7,NOR,MEDIUM,2.490657
8,NOR,MEDIUM,0.503550
9,NOR,SOFT,1.886200


In [9]:
# --- Summarize per compound ---
def summarize_degradation(deg_df):
    if deg_df.empty:
        return pd.DataFrame()

    # Pivot into columns for Soft/Medium/Hard
    pivot = deg_df.pivot_table(index='Driver', columns='Compound', values='DegRate', aggfunc='mean')
    pivot = pivot.rename(columns=lambda x: f'Deg_{x}').reset_index()

    for comp in ['Soft', 'Medium', 'Hard']:
        col = f'deg_{comp}'
        
    return pivot

In [10]:
summ_deg = summarize_degradation(tire_deg)
summ_deg

Compound,Driver,Deg_HARD,Deg_MEDIUM,Deg_SOFT
0,ALB,NaN,0.513319,1.831200
1,ALO,NaN,0.710963,2.953000
2,BOT,NaN,0.062168,NaN
3,GAS,NaN,0.712368,0.000000
4,HAM,NaN,1.082667,0.000000
5,HUL,NaN,3.732668,0.619750
6,LEC,NaN,1.048262,0.000000
7,MAG,NaN,4.214171,0.096057
8,NOR,NaN,1.497104,1.886200
9,OCO,NaN,0.200661,0.362612


#### Extracting Qualifying Data

In [51]:
def get_qualifying_data(season, gp_name):
    quali = fastf1.get_session(season, gp_name, 'Q')
    quali.load()

    results = quali.results
    results['Q1Time'] = results['Q1'].dt.total_seconds()
    results['Q2Time'] = results['Q2'].dt.total_seconds()
    results['Q3Time'] = results['Q3'].dt.total_seconds()
    results['Driver'] = results['Abbreviation']
    results['GridPos'] = results['Position'].astype('int')
    
        
    return results[['Driver', 'GridPosition', 'Q1Time', 'Q2Time', 'Q3Time']]

In [52]:
quali_df = get_qualifying_data(season, gp_name)
quali_df

core           INFO 	Loading data for Monaco Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver  3: Lap timing integrity check failed for 1 lap(s)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '81', '55', '4', '63', '1', '44', '22', '23', '10', '31', '3', '18', '27', '14', '2', '20', '11', '77', '24']


,Driver,GridPosition,Q1Time,Q2Time,Q3Time
16,LEC,NaN,71.584,70.825,70.270
81,PIA,NaN,71.500,70.756,70.424
55,SAI,NaN,71.543,71.075,70.518
4,NOR,NaN,71.760,70.732,70.542
63,RUS,NaN,71.492,70.929,70.543
1,VER,NaN,71.711,70.745,70.567
44,HAM,NaN,71.528,71.056,70.621
22,TSU,NaN,71.852,71.106,70.858
23,ALB,NaN,71.623,71.216,70.948
10,GAS,NaN,71.714,70.896,71.311


In [48]:
quali_df.info()

<class 'fastf1.core.SessionResults'>
Index: 20 entries, 16 to 24
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   drv           20 non-null     object 
 1   GridPosition  20 non-null     int64  
 2   Q1Time        20 non-null     float64
 3   Q2Time        15 non-null     float64
 4   Q3Time        10 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 960.0+ bytes


#### Extracting Race Result Label Data

In [56]:
def get_race_labels(season, gp_name):
    race = fastf1.get_session(season, gp_name, 'R')
    race.load()

    results = race.results[['Abbreviation', 'Position']].rename(
        columns={'Abbreviation': 'Driver', 'Position': 'RaceResult'}
    )
    
        
    return results[['Driver', 'RaceResult']]

#### Merge All Features + Race Result

In [57]:
def build_features(season, gp_name):
    # session = fastf1.get_session(season, gp_name, 'FP2')
    # session.load()

    pace_data = get_practice_data(season, gp_name)
    quali_data =  get_qualifying_data(season, gp_name)
    race_labels = get_race_labels(season, gp_name)

    # Merging the data
    df = pace_data.merge(quali_data, on='Driver', how='left')
    df = df.merge(race_labels, on='Driver', how='left')

    return df

features_df = build_features(2024, 'Bahrain')

core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '2', '3', '4', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '31', '44', '55', '63', '77', '81']
core           INFO 	Loading data for Bahrain Grand Prix - Practice 2 [v3.5.3]
req            INFO 	Using cac

In [59]:
features_df

,Driver,FP1_FastestLap,FP1_AvgLapTime,FP1_NumLaps,FP2_FastestLap,FP2_AvgLapTime,FP2_NumLaps,FP3_FastestLap,FP3_AvgLapTime,FP3_NumLaps,GridPosition,Q1Time,Q2Time,Q3Time,RaceResult
0,VER,93.238,96.409777,9,90.851,94.751111,9,91.062,92.142250,4,NaN,90.031,89.374,89.179,1.0
1,SAR,94.213,95.906111,9,91.715,92.889833,6,92.125,92.732000,4,NaN,90.770,NaN,NaN,20.0
2,RIC,92.869,96.718181,11,91.516,92.317750,4,91.449,92.110500,2,NaN,90.562,90.278,NaN,13.0
3,NOR,92.901,96.523090,11,92.608,92.754000,2,91.118,92.163333,3,NaN,90.143,89.941,89.614,6.0
4,GAS,95.144,95.739166,6,91.951,92.304250,4,92.382,93.026500,4,NaN,90.948,NaN,NaN,18.0
5,PER,93.413,96.357090,11,91.115,93.751500,6,91.248,92.328000,4,NaN,90.221,89.932,89.537,2.0
6,ALO,93.193,93.897500,6,90.660,91.209666,3,90.965,92.433600,5,NaN,90.179,89.801,89.542,9.0
7,LEC,93.268,96.301900,10,91.113,93.425400,5,91.094,92.518666,6,NaN,90.243,89.165,89.407,4.0
8,STR,93.868,94.804571,7,90.891,91.688500,4,91.396,92.182600,5,NaN,89.965,90.200,NaN,10.0
9,MAG,97.477,98.346437,16,91.764,92.109250,4,91.671,94.610000,8,NaN,90.646,90.529,NaN,12.0
